In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from dask.dataframe.methods import drop_columns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.impute import SimpleImputer,KNNImputer
from sklearn.metrics import mean_absolute_error

In [ ]:
titanic = sns.load_dataset("titanic")
print(titanic.shape)
titanic.head()

In [ ]:
print(titanic.isna().sum())
sns.heatmap(titanic.isna(), cbar=False)
plt.title("Missing Data Heatmap")
plt.show()

In [ ]:
drop_rows = titanic.dropna(subset=['age'])
drop_columns = titanic.drop(columns=['deck'])
drop_threst = titanic.dropna(thresh=14)

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(10,4))
sns.histplot(titanic['age'],bins=30,kde=True,ax=ax[0])
ax[0].set_title("Original age dist")
sns.histplot(drop_rows['age'],bins=30,kde=True,ax=ax[1])
ax[1].set_title("After Deletion")
plt.show()

In [ ]:
df_del = titanic.dropna()
print("After Deletion Shape:", df_del.shape)

In [ ]:
imp_mean = SimpleImputer(strategy="mean")
titanic['age_mean'] = imp_mean.fit_transform(titanic[['age']])

imp_median = SimpleImputer(strategy="median")
titanic['age_median'] = imp_median.fit_transform(titanic[['age']])

imp_mode = SimpleImputer(strategy="most_frequent")
titanic['age_mode'] = imp_mode.fit_transform(titanic[['age']])

In [ ]:
titanic['age_ffill'] = titanic['age'].fillna(method='ffill')
titanic['age_bfill'] = titanic['age'].fillna(method='bfill')

In [ ]:
knn = KNNImputer(n_neighbors=5)
titanic['age_knn'] = knn.fit_transform(titanic[['age','fare']])[:,0]

plt.figure(figsize=(10,5))
sns.kdeplot(titanic['age'], label='Original (with NaN)')
sns.kdeplot(titanic['age_mean'], label='Mean Imputed')
sns.kdeplot(titanic['age_median'], label='Median Imputed')
sns.kdeplot(titanic['age_knn'], label='KNN Imputed')
plt.legend(); plt.title("Age Imputation Comparison"); plt.show()

In [ ]:
# Apply linear interpolation on 'age'
titanic['age_linear'] = titanic['age'].interpolate(method='linear')
# Check before vs after for some rows with missing values
print(titanic.loc[titanic['age'].isna(), ['age','age_linear']].head(10))

In [ ]:
# Polynomial interpolation (order 2 = quadratic)
titanic['age_poly'] = titanic['age'].interpolate(method='polynomial', order=2)
# Spline interpolation (cubic smoothing)
titanic['age_spline'] = titanic['age'].interpolate(method='spline', order=3)

In [ ]:
plt.figure(figsize=(12,6))
sns.kdeplot(titanic['age'], label='Original (with NaN)')
sns.kdeplot(titanic['age_linear'], label='Linear')
sns.kdeplot(titanic['age_poly'], label='Polynomial (deg=2)')
sns.kdeplot(titanic['age_spline'], label='Spline (deg=3)')
plt.legend()
plt.title("Age Imputation: Interpolation Methods")
plt.show()

In [ ]:
from scipy.stats import zscore

titanic['fare_z'] = zscore(titanic['fare'].fillna(0))
outliers_z = titanic[np.abs(titanic['fare_z']) > 3]
print("Z-score outliers in fare:", outliers_z.shape[0])

In [ ]:
plt.figure(figsize=(10,4))
sns.boxplot(x=titanic['fare'])
plt.title("Boxplot of Fare (Outliers appear as distant points)")
plt.show()

In [ ]:
plt.figure(figsize=(10,5))
plt.scatter(range(len(titanic)), titanic['fare'], label="Data", alpha=0.6)
plt.scatter(outliers_z.index, outliers_z['fare'], color='red', label="Z-score Outliers", marker='x', s=60)
plt.xlabel("Passenger Index")
plt.ylabel("Fare")
plt.title("Fare with Z-score Outliers Highlighted")
plt.legend()
plt.show()